In [35]:
from gurobipy import GRB, quicksum
import gurobipy as gb

In [36]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [37]:
df = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/updated_gym_data.csv')

In [38]:
df.head(10)

,Exercise,Category,BodyPart,Equipment,Difficulty,Stimulus-to-Fatigue,Expected Time,Hypertrophy Rating
0,Bench Press With Short Bands,Powerlifting,Chest,Bands,Beginner,0.817884,15.518089,0.596124
1,Hip Lift with Band,Powerlifting,Glutes,Bands,Beginner,0.768902,14.655351,0.623237
2,Band Good Morning (Pull Through),Powerlifting,Hamstrings,Bands,Beginner,0.792188,16.292358,0.601159
3,Speed Box Squat,Powerlifting,Quadriceps,Bands,Intermediate,0.599044,17.109781,0.800347
4,Partner plank band row,Strength,Abdominals,Bands,Intermediate,0.730726,14.212727,0.461565
5,Banded crunch isometric hold,Strength,Abdominals,Bands,Intermediate,0.716055,12.619634,0.416539
6,FYR Banded Plank Jack,Strength,Abdominals,Bands,Intermediate,0.766785,12.604063,0.534897
7,Banded crunch,Strength,Abdominals,Bands,Intermediate,0.765022,12.996750,0.464355
8,Crunch,Strength,Abdominals,Bands,Intermediate,0.710541,14.824696,0.494665
9,Decline band press sit-up,Strength,Abdominals,Bands,Intermediate,0.764133,13.953021,0.437804


In [39]:
df.columns

Index(['Exercise', 'Category', 'BodyPart', 'Equipment', 'Difficulty',
       'Stimulus-to-Fatigue', 'Expected Time', 'Hypertrophy Rating'],
      dtype='object')

In [40]:
df['BodyPart'].unique()

array(['Chest', 'Glutes', 'Hamstrings', 'Quadriceps', 'Abdominals',
       'Adductors', 'Abductors', 'Biceps', 'Calves', 'Forearms', 'Lats',
       'Lower Back', 'Middle Back', 'Traps', 'Shoulders', 'Triceps',
       'Neck'], dtype=object)

In [41]:
#getting data from data set
exercise_ids = df.index
hypertrophy_ratings = df['Hypertrophy Rating']
sfr_values = df['Stimulus-to-Fatigue']
body_parts = df['BodyPart']
categories = df['Category']
equipment_types = df['Equipment']
Difficulty_rating = df['Difficulty']

## Model

In [42]:
# Create a new optimization model to maximize profit
model = gb.Model("Hypertrophy Optimization")

In [43]:
# Decision variables
num_exercises = len(df)
x = model.addVars(num_exercises, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="ExerciseProportion")

In [44]:
# Objective function: Maximizes the total hypertrophy rating across all exercises based on their proportions.

model.setObjective(
        quicksum(df.loc[i, 'Hypertrophy Rating'] * x[i] for i in range(num_exercises)),
        GRB.MAXIMIZE
    )


In [45]:
# Adding the constraints

## 1. Limit the proportion per exercise to a maximum of 5%.
for i in range(num_exercises):
    model.addConstr(x[i] <= 0.05, name=f"MaxProportion_{i}")

# 2. Enforce minimum allocations for specific and general body parts.
general_min_allocation = 0.025  # 2.5%
specific_min_allocations = {
    'Traps': 0.005,      # 0.5%
    'Neck': 0.005,       # 0.5%
    'Forearms': 0.005,   # 0.5%
    'Abdominals': 0.04   # 4%
}

# General and specific allocation constraints for body parts
unique_body_parts = df['BodyPart'].unique()
for part in unique_body_parts:
    if part in specific_min_allocations:
        model.addConstr(
            gb.quicksum(x[i] for i in exercise_ids if body_parts[i] == part) >= specific_min_allocations[part],
            f"{part}Min"
        )
    else:
        model.addConstr(
            gb.quicksum(x[i] for i in exercise_ids if body_parts[i] == part) >= general_min_allocation,
            f"{part}GeneralMin"
        )

# 3. Ensure leg muscles allocation is at least 2.6 times the upper body allocation.
leg_muscles = ['Adductors', 'Abductors', 'Calves', 'Glutes', 'Hamstrings', 'Quadriceps']
upper_body_muscles = ['Chest', 'Lower Back', 'Middle Back', 'Biceps', 'Traps', 'Triceps', 
                      'Shoulders', 'Abdominals', 'Forearms', 'Neck', 'Lats']

leg_upper_body_ratio_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if body_parts[i] in leg_muscles) >= 
    2.6 * gb.quicksum(x[i] for i in exercise_ids if body_parts[i] in upper_body_muscles),
    "LegUpperBodyRatio"
)

# 4. Balance biceps and triceps allocations with chest, lower back, and middle back allocations.
muscle_group_balance_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if body_parts[i] in ['Biceps', 'Triceps']) ==
    gb.quicksum(x[i] for i in exercise_ids if body_parts[i] in ['Chest', 'Lower Back', 'Middle Back']),
    "MuscleGroupBalance"
)

# 5. Restrict the overall Stimulus-to-Fatigue Ratio (SFR) to a maximum of 0.55
sfr_constraint = model.addConstr(
    gb.quicksum(sfr_values[i] * x[i] for i in exercise_ids) <= 0.55,
    "SFRConstraint"
)

# 6. Maintain beginner ≥ 1.4 × intermediate ≥ advanced difficulty ratios.
beginner_intermediate_ratio_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if Difficulty_rating[i] == 'Beginner') >= 
    1.4 * gb.quicksum(x[i] for i in exercise_ids if Difficulty_rating[i] == 'Intermediate'),
    "BeginnerIntermediateRatio"
)

intermediate_advanced_ratio_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if Difficulty_rating[i] == 'Intermediate') >= 
    1.1 * gb.quicksum(x[i] for i in exercise_ids if Difficulty_rating[i] == 'Advanced'),
    "IntermediateAdvancedRatio"
)

# 7. Set minimum and maximum allocations for Strongman, Powerlifting, and Olympic Weightlifting exercises.
# Strongman exercises ≤ 8%
strongman_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if categories[i] == 'Strongman') <= 0.08,
    "StrongmanMax"
)

# Powerlifting exercises ≥ 9%
powerlifting_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if categories[i] == 'Powerlifting') >= 0.09,
    "PowerliftingMin"
)

# Olympic Weightlifting exercises ≥ 10%
olympic_weightlifting_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if categories[i] == 'Olympic Weightlifting') >= 0.10,
    "OlympicWeightliftingMin"
)

# 8. Ensure at least 60% of exercises involve essential equipment types.
essential_equipment = ['Barbell', 'Dumbbell', 'Machine', 'Cable', 'E-Z Curl Bar', 'Bands']
equipment_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if equipment_types[i] in essential_equipment) >= 0.6,
    "EssentialEquipmentMin"
)

# Ensure total proportion is 1
model.addConstr(gb.quicksum(x[i] for i in range(len(df))) == 1, "Total_Proportion")

<gurobi.Constr *Awaiting Model Update*>

In [46]:
# Solve the optimization model to find the optimal proportions for each exercise while adhering to all constraints.
# The model successfully converges to an optimal solution with an objective value of 0.7672, maximizing hypertrophy.
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2664 rows, 2637 columns and 20237 nonzeros
Model fingerprint: 0x25a0cea5
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 2638 rows and 0 columns
Presolve time: 0.01s
Presolved: 26 rows, 2637 columns, 15326 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   2.805000e+00   0.000000e+00      0s
      31    7.6726674e-01   0.000000e+00   0.000000e+00      0s

Solved in 31 iterations and 0.01 seconds (0.01 work units)
Optimal objective  7.672667395e-01


In [47]:
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Computing IIS...")
    model.computeIIS()
    model.write("infeasible_constraints.ilp")

In [48]:
# Check the optimization status
if model.status == GRB.OPTIMAL:
    print("The optimal solution: ", model.objVal)
    # Optionally, display the decision variables
    for i in exercise_ids:
        if x[i].x > 0:
            print(f"Exercise {df.loc[i, 'Exercise']}: Proportion = {x[i].x:.4f}")
elif model.status == GRB.INFEASIBLE:
    print("The model is infeasible.")
elif model.status == GRB.UNBOUNDED:
    print("The model is unbounded.")
else:
    print("Optimization was stopped with status", model.status)

The optimal solution:  0.7672667395203319
Exercise Clean Deadlift: Proportion = 0.0500
Exercise Muscle Snatch: Proportion = 0.0500
Exercise Clean Shrug: Proportion = 0.0050
Exercise Clean from Blocks: Proportion = 0.0500
Exercise Power snatch-: Proportion = 0.0500
Exercise Split Jerk: Proportion = 0.0500
Exercise Heaving Snatch Balance: Proportion = 0.0103
Exercise Board bench press: Proportion = 0.0250
Exercise Barbell glute bridge: Proportion = 0.0500
Exercise Barbell Hip Thrust: Proportion = 0.0308
Exercise Sumo deadlift: Proportion = 0.0500
Exercise Good Morning: Proportion = 0.0500
Exercise Good Morning off Pins: Proportion = 0.0500
Exercise Hanging Bar Good Morning: Proportion = 0.0288
Exercise Rack pull: Proportion = 0.0250
Exercise Barbell back squat to box: Proportion = 0.0500
Exercise Reverse Band Power Squat: Proportion = 0.0500
Exercise Pin Presses: Proportion = 0.0500
Exercise 30 Barbell Floor Wiper: Proportion = 0.0400
Exercise Smith machine standing calf raise: Proportio

## d and e 

### Shadow Prices

In [49]:
#geting the shadow prices of the model 
# Loop through all constraints and print their shadow prices
for constr in model.getConstrs():
    print(f"Constraint {constr.ConstrName}: Shadow Price = {constr.Pi}")

Constraint MaxProportion_0: Shadow Price = 0.0
Constraint MaxProportion_1: Shadow Price = 0.0
Constraint MaxProportion_2: Shadow Price = 0.0
Constraint MaxProportion_3: Shadow Price = 0.0
Constraint MaxProportion_4: Shadow Price = 0.0
Constraint MaxProportion_5: Shadow Price = 0.0
Constraint MaxProportion_6: Shadow Price = 0.0
Constraint MaxProportion_7: Shadow Price = 0.0
Constraint MaxProportion_8: Shadow Price = 0.0
Constraint MaxProportion_9: Shadow Price = 0.0
Constraint MaxProportion_10: Shadow Price = 0.0
Constraint MaxProportion_11: Shadow Price = 0.0
Constraint MaxProportion_12: Shadow Price = 0.0
Constraint MaxProportion_13: Shadow Price = 0.0
Constraint MaxProportion_14: Shadow Price = 0.0
Constraint MaxProportion_15: Shadow Price = 0.0
Constraint MaxProportion_16: Shadow Price = 0.0
Constraint MaxProportion_17: Shadow Price = 0.0
Constraint MaxProportion_18: Shadow Price = 0.0
Constraint MaxProportion_19: Shadow Price = 0.0
Constraint MaxProportion_20: Shadow Price = 0.0
Co

## Model with Relaxed SPR

In [50]:
# Create a new optimization model to maximize profit
s_model = gb.Model("Hypertrophy Optimization with SHR")


In [51]:
#decision variables 
x = s_model.addVars(exercise_ids, lb=0, ub=0.05, name="x")

In [52]:
#objective function
s_model.setObjective(gb.quicksum(hypertrophy_ratings[i] * x[i] for i in exercise_ids), GRB.MAXIMIZE)

In [53]:
#adding the constraints
# 1. Total allocation must equal 1 (100% of the workout)
total = s_model.addConstr(gb.quicksum(x[i] for i in exercise_ids) == 1, "TotalAllocation")

# 2. SFR constraint: overall SFR ratio ≤ 0.55
SFR_constraint = s_model.addConstr(gb.quicksum(sfr_values[i] * x[i] for i in exercise_ids) <= 0.551, "SFRConstraint")


# 4. Leg = 2.6 × Upper Body
leg_muscles = ['Adductors', 'Abductors', 'Calves', 'Glutes', 'Hamstrings', 'Quadriceps']
upper_body = ['Chest', 'Lower Back','Middle Back', 'Biceps','Traps', 'Triceps', 'Shoulders','Abdominals','Forearms','Neck','Lats']
leg_constraint = s_model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if body_parts[i] in leg_muscles) >= 
    2.6 * gb.quicksum(x[i] for i in exercise_ids if body_parts[i] in upper_body), 
    "LegUpperBodyRatio"
)

#biceps and triceps
MuscleGroupBalance_constraint = s_model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if body_parts[i] in ['Biceps', 'Triceps']) ==
    gb.quicksum(x[i] for i in exercise_ids if body_parts[i] in ['Chest', 'Lower Back', 'Middle Back']),
    "MuscleGroupBalance"
)

#body allocations

#3. Body part minimum allocations
general_min_allocation = 0.025  # 2.5%
specific_min_allocations = {
    'Traps': 0.005,  # 0.5%
    'Neck': 0.005,  # 0.5%
    'Forearms': 0.005,  # 0.5%
    'Abdominals': 0.04  # 4%
}
# General allocation for all body parts
unique_body_parts = df['BodyPart'].unique()
for part in unique_body_parts:
    if part in specific_min_allocations:
        s_model.addConstr(
            gb.quicksum(x[i] for i in exercise_ids if body_parts[i] == part) >= specific_min_allocations[part],
            f"{part}Min"
        )
    else:
        s_model.addConstr(
            gb.quicksum(x[i] for i in exercise_ids if body_parts[i] == part) >= 0.025,
            f"{part}GeneralMin"
        )
#Strongman exercoes
Strongman = s_model.addConstr(gb.quicksum(x[i] for i in exercise_ids if categories[i] == 'Strongman') <= 0.08, "StrongmanMax")

#powerlifting
Powerlifting = s_model.addConstr(gb.quicksum(x[i] for i in exercise_ids if categories[i] == 'Powerlifting') >= 0.09, "PowerliftingMin") 

#Olympic Weightlifting 
Olympic_weightlifting = s_model.addConstr(gb.quicksum(x[i] for i in exercise_ids if categories[i] == 'Olympic Weightlifting') >= 0.10, "OlympicWeightliftingMin") 

#8. Equipment-based exercises ≥ 60%
essential_equipment = ['Barbell', 'Dumbbell', 'Machine', 'Cable', 'E-Z Curl Bar', 'Bands']
equipment = s_model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if equipment_types[i] in essential_equipment) >= 0.6, 
    "EssentialEquipmentMin"
)

# 6. Beginner ≥ 1.4 × Intermediate ≥ Advanced
Beginner = s_model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if Difficulty_rating[i] == 'Beginner') >= 
    1.4 * gb.quicksum(x[i] for i in exercise_ids if Difficulty_rating[i] == 'Intermediate'),
    "BeginnerIntermediateRatio"
)
Intermediate = s_model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if Difficulty_rating[i] == 'Intermediate') >= 
    1.1 * gb.quicksum(x[i] for i in exercise_ids if Difficulty_rating[i] == 'Advanced'),
    "IntermediateAdvancedRatio"
)


In [54]:
# Solve our model
s_model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 27 rows, 2637 columns and 17600 nonzeros
Model fingerprint: 0x282d1506
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [5e-02, 5e-02]
  RHS range        [5e-03, 1e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 26 rows, 2637 columns, 15326 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   2.805000e+00   0.000000e+00      0s
      31    7.6864373e-01   0.000000e+00   0.000000e+00      0s

Solved in 31 iterations and 0.00 seconds (0.01 work units)
Optimal objective  7.686437338e-01


In [55]:
# Check the optimization status
if s_model.status == GRB.OPTIMAL:
    print("The optimal solution: ", s_model.objVal)
    # Optionally, display the decision variables
    for i in exercise_ids:
        if x[i].x > 0:
            print(f"Exercise {df.loc[i, 'Exercise']}: Proportion = {x[i].x:.4f}")
elif s_model.status == GRB.INFEASIBLE:
    print("The model is infeasible.")
elif s_model.status == GRB.UNBOUNDED:
    print("The model is unbounded.")
else:
    print("Optimization was stopped with status", s_model.status)

The optimal solution:  0.7686437337511811
Exercise Clean Deadlift: Proportion = 0.0500
Exercise Muscle Snatch: Proportion = 0.0500
Exercise Clean Shrug: Proportion = 0.0050
Exercise Clean from Blocks: Proportion = 0.0500
Exercise Power snatch-: Proportion = 0.0500
Exercise Split Jerk: Proportion = 0.0500
Exercise Heaving Snatch Balance: Proportion = 0.0046
Exercise Board bench press: Proportion = 0.0250
Exercise Barbell glute bridge: Proportion = 0.0500
Exercise Barbell Hip Thrust: Proportion = 0.0308
Exercise Sumo deadlift: Proportion = 0.0500
Exercise Good Morning: Proportion = 0.0500
Exercise Good Morning off Pins: Proportion = 0.0500
Exercise Hanging Bar Good Morning: Proportion = 0.0345
Exercise Rack pull: Proportion = 0.0250
Exercise Barbell back squat to box: Proportion = 0.0500
Exercise Reverse Band Power Squat: Proportion = 0.0500
Exercise Pin Presses: Proportion = 0.0500
Exercise 30 Barbell Floor Wiper: Proportion = 0.0400
Exercise Smith machine standing calf raise: Proportio

###  Part (f): Including Barbell Back Squats

In [56]:

exercise_name = "Barbell Back Squats"
exercise_index = df[df['Exercise'] == exercise_name].index[0]

variable = x[exercise_index]

if model.status == GRB.OPTIMAL:
    print(f"Sensitivity Information for '{exercise_name}':")
    print(f"  Current Hypertrophy Rating: {df['Hypertrophy Rating'][exercise_index]:.4f}")
    print(f"  Increase Limit (SAObjUp): {variable.SAObjUp:.4f}")
    print(f"  Decrease Limit (SAObjLow): {variable.SAObjLow:.4f}")
else:
    print("Model is not optimized.")

Sensitivity Information for 'Barbell Back Squats':
  Current Hypertrophy Rating: 0.6783
  Increase Limit (SAObjUp): 1.1491
  Decrease Limit (SAObjLow): -inf


### Part(h): Optimal Solution with Relaxed Constraints

In [68]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

def optimize_with_reduced_constraints(df):
    """
    Optimizes the hypertrophy rating with reduced constraints,
    ensuring alignment with the dual problem formulation.
    """
    model = Model("Reduced Constraints Optimization")
    model.setParam('OutputFlag', 1)  # Enable Gurobi's detailed output

    # Decision variables
    num_exercises = len(df)
    x = model.addVars(num_exercises, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="ExerciseProportion")

    # Objective: Maximize hypertrophy rating
    model.setObjective(
        quicksum(df['Hypertrophy Rating'].iloc[i] * x[i] for i in range(num_exercises)),
        GRB.MAXIMIZE
    )

    # Constraint 1: Maximum proportion per exercise
    for i in range(num_exercises):
        model.addConstr(x[i] <= 0.05, name=f"MaxProportion_{i}")

    # Constraint 2: Minimum proportions for specific muscle groups
    muscle_group_thresholds = {"Traps": 0.005, "Neck": 0.005, "Forearms": 0.005, "Abdominals": 0.04}
    for group, threshold in muscle_group_thresholds.items():
        indices = df[df['BodyPart'] == group].index.to_list()
        model.addConstr(quicksum(x[i] for i in indices) >= threshold, name=f"MinProportion_{group}")

    # Constraint 2.5%: General threshold for other body parts
    exceptions = set(muscle_group_thresholds.keys())
    all_body_parts = set(df['BodyPart'].unique())
    remaining_body_parts = all_body_parts - exceptions

    for body_part in remaining_body_parts:
        indices = df[df['BodyPart'] == body_part].index.to_list()
        model.addConstr(quicksum(x[i] for i in indices) >= 0.025, name=f"MinProportion_{body_part}")

    # Constraint 8: Equipment proportion requirement
    equipment_types = ["Barbell", "Dumbbell", "Machine", "Cable", "E-Z Curl Bar", "Bands"]
    equipment_indices = df[df['Equipment'].isin(equipment_types)].index.to_list()
    model.addConstr(quicksum(x[i] for i in equipment_indices) >= 0.60, name="EquipmentProportion")

    # Normalize proportions to sum to 1
    model.addConstr(quicksum(x[i] for i in range(num_exercises)) == 1, name="TotalProportion")

    # Optimize the model
    model.optimize()

# Output the results
if optimal_rating is not None:
    print(f"Optimal Hypertrophy Rating: {optimal_rating}")
    print("Exercise Allocation:")
    for exercise, proportion in sorted(exercise_allocation.items(), key=lambda x: x[1], reverse=True):
        if proportion > 0:
            print(f"{exercise}: {proportion:.2%}")

Optimal Hypertrophy Rating: 0.8570357585549997
Exercise Allocation:
Behind-the-head push-press: 5.00%
Board bench press: 5.00%
Barbell glute bridge: 5.00%
Barbell Hip Thrust: 5.00%
Good Morning: 5.00%
Sumo Deadlift with Bands: 5.00%
Good Morning off Pins: 5.00%
Hanging Bar Good Morning: 5.00%
Reverse Band Sumo Deadlift: 5.00%
Rack pull: 5.00%
Seated Good Mornings: 5.00%
Barbell back squat to box: 5.00%
Bench Press - Powerlifting: 5.00%
Dumbbell floor press: 5.00%
Glute Ham Raise: 5.00%
Bench Press with Chains: 4.50%
KV Barbell Hip Thrust: 4.00%
EZ-bar spider curl: 2.50%
Barbell seal row: 2.50%
Cross-over jack: 2.50%
Behind-The-Neck Pull-Down - Gethin Variation: 2.50%
Standing Dumbbell Calf Raise: 2.50%
Thigh adductor: 2.50%
Single-arm inverted row: 0.50%
Isometric Neck Exercise - Front And Back: 0.50%
Dumbbell farmer's walk: 0.50%


### Part(i): Formulating and Solving the Dual Problem

In [66]:
from gurobipy import Model, GRB, quicksum

def solve_dual(df):
    """
    Solve the dual problem for the primal optimization.
    """
    # Create the dual model
    dual_model = Model("Dual_Problem")

    # Primal data
    num_exercises = len(df)
    muscle_groups = {"Traps", "Neck", "Forearms", "Abdominals"}
    equipment_types = ["Barbell", "Dumbbell", "Machine", "Cable", "E-Z Curl Bar", "Bands"]

    # Define muscle group thresholds from the primal
    muscle_thresholds = {
        "Traps": 0.005,
        "Neck": 0.005,
        "Forearms": 0.005,
        "Abdominals": 0.04
    }
    default_body_part_weight = 0.025

    # Dual variables
    mu = dual_model.addVars(num_exercises, lb=0, vtype=GRB.CONTINUOUS, name="mu")
    nu = dual_model.addVars(muscle_groups, lb=0, vtype=GRB.CONTINUOUS, name="nu")
    lambda_var = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="lambda")
    sigma = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="sigma")

    # Objective function: Minimize
    dual_model.setObjective(
        quicksum(0.05 * mu[i] for i in range(num_exercises)) +
        quicksum(nu[g] * muscle_thresholds[g] for g in muscle_groups) +
        lambda_var * 0.60 +
        sigma,
        GRB.MINIMIZE
    )

    # Dual constraints
    for i in range(num_exercises):
        body_part = df['BodyPart'].iloc[i]
        equipment = df['Equipment'].iloc[i]
        hypertrophy_rating = df['Hypertrophy Rating'].iloc[i]

        dual_model.addConstr(
            mu[i] +
            (nu[body_part] if body_part in muscle_groups else 0) +
            (lambda_var if equipment in equipment_types else 0) +
            sigma >= hypertrophy_rating,
            name=f"Dual_Constraint_{i}"
        )

    # Optimize the dual problem
    dual_model.optimize()

    # Output results
    if dual_model.status == GRB.OPTIMAL:
        print("Optimization completed successfully.")
        print(f"Optimal Dual Objective Value: {dual_model.objVal:.4f}")

        # Dual Variables
        print("\nDual Variables:")
        for v in dual_model.getVars():
            print(f"{v.varName}: {v.x:.4f}")

        # Constraint Slackness
        print("\nConstraint Slackness:")
        for constr in dual_model.getConstrs():
            print(f"{constr.constrName}: Slack = {constr.slack:.4f}")

    return dual_model.objVal
# Run the dual problem solver
dual_score = solve_dual(df)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2637 rows, 2643 columns and 7283 nonzeros
Model fingerprint: 0x14231c71
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-03, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 1e+00]
Presolve removed 696 rows and 703 columns
Presolve time: 0.00s
Presolved: 1941 rows, 1940 columns, 5043 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.101312e+03   0.000000e+00      0s
      23    8.3749717e-01   0.000000e+00   0.000000e+00      0s

Solved in 23 iterations and 0.00 seconds (0.00 work units)
Optimal objective  8.374971723e-01
Optimization completed successfully.
Optimal Dual Objective Value: 0.8375

Dual Variables:
mu[0]: 0.0000
mu[1]: 0.0000
mu[2]: 0.0000
mu[3]: 0.0000
mu[4]: 0.0000
mu

In [84]:
from gurobipy import Model, GRB, quicksum

def solve_dual(df):
    """
    Solve the dual problem for the primal optimization with corrected alignment.
    """
    # Ensure no normalization distorts the objective
    df['Hypertrophy Rating'] = df['Hypertrophy Rating']

    # Create the dual model
    dual_model = Model("Dual_Problem")

    # Primal data
    num_exercises = len(df)
    muscle_groups = {"Traps", "Neck", "Forearms", "Abdominals"}
    equipment_types = ["Barbell", "Dumbbell", "Machine", "Cable", "E-Z Curl Bar", "Bands"]

    # Thresholds from the primal
    muscle_thresholds = {"Traps": 0.005, "Neck": 0.005, "Forearms": 0.005, "Abdominals": 0.04}

    # Dual variables
    mu = dual_model.addVars(num_exercises, lb=0, vtype=GRB.CONTINUOUS, name="mu")  # For max proportion
    nu = dual_model.addVars(muscle_groups, lb=0, vtype=GRB.CONTINUOUS, name="nu")  # For muscle groups
    lambda_var = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="lambda")      # For equipment
    sigma = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="sigma")            # For total proportion

    # Objective: Minimize
    dual_model.setObjective(
        quicksum(0.05 * mu[i] for i in range(num_exercises)) +   # Max proportion weight
        quicksum(nu[g] * muscle_thresholds[g] for g in muscle_groups) +  # Muscle group thresholds
        lambda_var * 0.60 +                                     # Equipment threshold
        sigma,                                                  # Total proportion
        GRB.MINIMIZE
    )

    # Dual constraints: Ensure alignment with primal constraints
    for i in range(num_exercises):
        hypertrophy_rating = df['Hypertrophy Rating'].iloc[i]
        body_part = df['BodyPart'].iloc[i]
        equipment = df['Equipment'].iloc[i]

        dual_model.addConstr(
            mu[i] +
            (nu[body_part] if body_part in muscle_groups else 0) +
            (lambda_var if equipment in equipment_types else 0) +
            sigma >= hypertrophy_rating,
            name=f"Dual_Constraint_{i}"
        )

    # Optimize the dual model
    dual_model.optimize()

    # Output results
    if dual_model.status == GRB.OPTIMAL:
        print("Optimization completed successfully.")
        print(f"Optimal Dual Objective Value: {dual_model.objVal:.10f}")  # Match primal precision

        # Log LHS, RHS, and gaps for each constraint
        print("\nConstraint Details:")
        for c in dual_model.getConstrs():
            constraint_name = c.constrName
            i = int(constraint_name.split('_')[-1])  # Extract exercise index from constraint name
            hypertrophy_rating = df['Hypertrophy Rating'].iloc[i]

            # Calculate LHS for the constraint
            lhs_value = (
                mu[i].x +
                (nu[df['BodyPart'].iloc[i]].x if df['BodyPart'].iloc[i] in muscle_groups else 0) +
                (lambda_var.x if df['Equipment'].iloc[i] in equipment_types else 0) +
                sigma.x
            )

            print(f"Constraint {constraint_name}: LHS = {lhs_value:.10f}, "
                  f"RHS = {hypertrophy_rating:.10f}, "
                  f"Gap = {lhs_value - hypertrophy_rating:.10f}")

    return dual_model.objVal

# Run the dual problem solver
dual_score = solve_dual(df)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2637 rows, 2643 columns and 7283 nonzeros
Model fingerprint: 0x14231c71
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-03, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 1e+00]
Presolve removed 696 rows and 703 columns
Presolve time: 0.00s
Presolved: 1941 rows, 1940 columns, 5043 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.101312e+03   0.000000e+00      0s
      23    8.3749717e-01   0.000000e+00   0.000000e+00      0s

Solved in 23 iterations and 0.00 seconds (0.00 work units)
Optimal objective  8.374971723e-01
Optimization completed successfully.
Optimal Dual Objective Value: 0.8374971723

Constraint Details:
Constraint Dual_Constraint_0: LHS = 0.8376828580, RHS = 0.5961